# Задача МГУ

Для начала работы, необходимо импортировать все зависимости и библиотеки

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

Создадим несколько функций для упрощения дальнейшей работы

In [ ]:
# function for debug
def printer(arg):
    with pd.option_context('display.max_columns', None):
        print(arg)


# function for changing True and False to 1 and 0
def replace_true_with_one(val):
    if val == True:
        return 1
    else:
        return 0


# function to get values between 0 and 1
def normalization(col):
    df[col] = df[col].apply(lambda lam: (lam - df[col].min()) / (df[col].max() - df[col].min()))
    return df[col]

Считываем данные

In [ ]:
df = pd.read_csv('data.csv')

Обработаем данные, заменяя единицами значения true и нулями значения false. Затем отбрасываем данные, которые не пригодятся.

In [ ]:
#change true and false with ones and zeros
df['shop_centre'] = df['shop_centre'].apply(replace_true_with_one, 1)
df['dns'] = df['dns'].apply(replace_true_with_one, 1)
df['is_center'] = df['is_center'].apply(replace_true_with_one, 1)
df['is_moscow'] = df['is_moscow'].apply(replace_true_with_one, 1)
#drop columns and none unique id
values_check = df.isnull().sum().sort_values(ascending=False).head()
df = df.drop(df['post_id'].nunique())
df = df.drop(columns=['lat', 'lon'])

Используя функцию нормализации данных (приведение к интервалу), обработаем столбцы таблицы

In [ ]:
# normalization of data
normalization(col='price')
normalization(col='parking_min_distance')
normalization(col='distance_100')
normalization(col='distance_500')
normalization(col='metro_min_distance')
normalization(col='transport_min_distance')

Запустите эту часть кода для просмотра фигуры корреляции.

In [ ]:
plt.figure(figsize=(16, 16))
plt.matshow(df.corr(), cmap='summer', fignum=1)
plt.colorbar()
plt.xticks(list(range(len(df.columns))), df.columns, rotation='vertical')
plt.yticks(list(range(len(df.columns))), df.columns, rotation='horizontal')
plt.show()

Проверка наибольших корреляций между значениями. В результате проверке, было обнаружено, что наиболее зависимые друг от друга величины - distance_500, distance_100, finance_count, price.

In [ ]:
correlation_price = df.corr()['price'].sort_values(ascending=False)
correlation_distance_500_count = df.corr()['distance_500'].sort_values(ascending=False)
correlation_distance_100_count = df.corr()['distance_100'].sort_values(ascending=False)

Создадим базу данных, дочернюю от исходной, которая включает только наиболее коррелируемые величины.

In [ ]:
# create sub df for further steps
df_temp = df[['id', 'distance_500', 'distance_100', 'price']]
df_temp = df_temp.sort_values('distance_500')
df_temp = df_temp[df_temp['price'].notna()]

Создадим кластеризацию модели. Во время выполнения этой части задания, было эмпирическим путем установлено, что наиболее подходящее число кластеров - 3

In [ ]:
X = df_temp.values[:, 1:]
X = np.nan_to_num(X)
k_means = KMeans(init="k-means++", n_clusters=3, n_init=12)
k_means.fit(X)
labels = k_means.labels_

Распечатаем кластеризированную модель

In [ ]:
# plot the clustering model
df_temp['cluster'] = labels
area = np.pi * (X[:, 1]) ** 2
plt.scatter(X[:, 0], X[:, 2], s=area, c=labels.astype(np.float), alpha=0.5)
plt.xlabel('price', fontsize=14)
plt.ylabel('distance_500', fontsize=14)
plt.show(

Из данного изображения видно, что существуют отделения банка, в которых значения price высокое относительно значения distance_500. Поэтому было принято решение ввести коэффициент, отражающий отношение price/distance_500. Учитывая, что данные нормализованы, - чем ниже значение distance_500, тем выше значение коэффициента. После вычисления коэффициента для каждой строки. Находится медиана коэффициента. Отделения банка, попавшие в значение коэффициента выше медианы - не выгодные. ниже - выгодные. Однако, необходимо также обратить внимание на конкуренцию, возможно рядом нет никаких организаций, кроме почта банка. В таком случае, нужно оставить отделение банка. Поэтому вводится ещё один коэффициент - медиана количества финансовых организаций в ближайшем окружении. 

In [ ]:
# hand-made coefficient= price/distance_500, which reflects income and outcome. Also shows the anomalies
df_temp['price_by_distance'] = df_temp['price'] / df_temp['distance_500']
median_of_pbd = df_temp['price_by_distance'].median()
median_of_finance_count = df['finance_count'].median()

При этом, если конкуренция низкая, но и проходимость низкая - нужно провести дополнительный анализ (значение=0,5). Если конкуренция высокая, проходимость низкая - необходимо закрыть отделение (значение=0). Если конкуренция высокая, проходимость высокая - оставить (значение=1). Если конкуренция низкая, проходимость высокая - оставить отделение (значение=1). где значение - hint в таблице.

In [ ]:
for index, row in df_temp.iterrows():
    if (df_temp.loc[index, 'price_by_distance'] > median_of_pbd and \
            df.loc[index, 'finance_count'] > median_of_finance_count):
        df_temp.loc[index, 'hint'] = 0
    elif (df_temp.loc[index, 'price_by_distance'] > median_of_pbd and\
            df.loc[index, 'finance_count']) < median_of_finance_count:
        df_temp.loc[index, 'hint'] = 0.5
    else:
        df_temp.loc[index, 'hint'] = 1

Сохранение полученной таблице в excel, если необходимо

In [ ]:
#  save the result in excel if needed
def saver(frame, save=False, path='output.xls'):
    if save:
        frame.to_excel(path=path)

по вызову функции, необходимо передать dataframe для сохранения. 